In [48]:
import sqlite3
import pandas as pd
import numpy as np
import sys

In [49]:
pd.__version__

'0.24.0'

In [50]:
MQ_FEATURE_ID = 8
feature_ids = [1299, 279, 4339, 1290, 2969, 4483, 4238, 42340, 35437, 111912, 134869]

In [51]:
import pickle

with open("/Users/darylwilding-mcbride/Downloads/full_mq_feature_{}.pickle".format(MQ_FEATURE_ID), "rb") as input_file:
    feature_df_list = pickle.load(input_file)

In [52]:
len(feature_df_list)

11

In [53]:
feature_df_list[2].head()

,frame_id,point_id,mz,scan,intensity,peak_id,raw_frame_point,retention_time_secs
0,41412,47019,878.390675,327,76,0,41412|47019,4447.901546
1,41412,47318,878.400022,328,52,0,41412|47318,4447.901546
2,41412,47319,878.904809,328,163,0,41412|47319,4447.901546
3,41412,47919,878.404695,330,96,0,41412|47919,4447.901546
4,41412,48251,878.390675,331,140,0,41412|48251,4447.901546


In [54]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]


In [55]:
for idx,feature_id in enumerate(feature_ids):
    df = feature_df_list[idx]
    df["hover"] = df["mz"].map('{:,.4f} m/z'.format) + ', ' + df["scan"].map('{} scan'.format) + ', feature {}'.format(feature_id)

In [56]:
feature_df_list[2].head()

,frame_id,point_id,mz,scan,intensity,peak_id,raw_frame_point,retention_time_secs,hover
0,41412,47019,878.390675,327,76,0,41412|47019,4447.901546,"878.3907 m/z, 327 scan, feature 4339"
1,41412,47318,878.400022,328,52,0,41412|47318,4447.901546,"878.4000 m/z, 328 scan, feature 4339"
2,41412,47319,878.904809,328,163,0,41412|47319,4447.901546,"878.9048 m/z, 328 scan, feature 4339"
3,41412,47919,878.404695,330,96,0,41412|47919,4447.901546,"878.4047 m/z, 330 scan, feature 4339"
4,41412,48251,878.390675,331,140,0,41412|48251,4447.901546,"878.3907 m/z, 331 scan, feature 4339"


In [57]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # requires version >= 1.9.0

3.5.0


In [58]:
init_notebook_mode(connected=True)

In [59]:
import plotly.plotly as py
import plotly.graph_objs as go

traces = []
for idx,feature_id in enumerate(feature_ids):
    df = feature_df_list[idx]
    if len(df) > 0:
        trace = go.Scatter3d(
            x = df.mz,
            y = df.scan,
            z = df.intensity,
            mode = 'markers',
            marker = dict(
                size = 5,
                color = "rgb{}".format(tableau20[idx]),
                opacity = 0.8,
                line = dict(
                    color = 'rgb(128, 128, 128)',
                    width = 0.1
                )
            ),
            text = df.hover,
            hoverinfo = 'text',
            name = "feature {}".format(feature_id)
        )
        traces.append((trace))


data = traces
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    scene = dict(
        xaxis = dict(
            title='m/z', autorange='reversed'),
        yaxis = dict(
            title='scan'),
        zaxis = dict(
            title='intensity')
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)


find the frame with the peak intensity

In [60]:
feature_of_interest = 2969
apex_frame_id = 0
for idx,feature_id in enumerate(feature_ids):
    df = feature_df_list[idx]
    if feature_id == feature_of_interest:
        apex_frame_id = df.iloc[df.intensity.idxmax()].frame_id

In [61]:
apex_frame_id

41500

In [63]:
feature_apex_frame_df_list = []
for idx,feature_id in enumerate(feature_ids):
    df = feature_df_list[idx]
    df = df[(df.frame_id == apex_frame_id)]
    feature_apex_frame_df_list.append((df))

In [64]:
import plotly.plotly as py
import plotly.graph_objs as go

traces = []
for idx,feature_id in enumerate(feature_ids):
    df = feature_apex_frame_df_list[idx]
    if len(df) > 0:
        trace = go.Scatter3d(
            x = df.mz,
            y = df.scan,
            z = df.intensity,
            mode = 'markers',
            marker = dict(
                size = 5,
                color = "rgb{}".format(tableau20[idx]),
                opacity = 0.8,
                line = dict(
                    color = 'rgb(128, 128, 128)',
                    width = 0.1
                )
            ),
            text = df.hover,
            hoverinfo = 'text',
            name = "feature {}".format(feature_id)
        )
        traces.append((trace))


data = traces
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    ),
    scene = dict(
        xaxis = dict(
            title='m/z', autorange='reversed'),
        yaxis = dict(
            title='scan'),
        zaxis = dict(
            title='intensity')
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)
